### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# 使用 colab 環境的同學請執行以下程式碼
# %tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
#import tensorflow as tf
#print(tf.__version__)
#
#import os
#from google.colab import drive 
#drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
#%cd 'gdrive/My Drive'
# os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
# %cd cupoy_cv_part4 # 可以自己改路徑

In [4]:
#%cd 'Colab Notebooks'
#%pwd

In [5]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [6]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [7]:
imgs_test, points_test = load_data(dirname = 'test.csv')
print("圖像資料:", imgs_test.shape, "\n關鍵點資料:", points_test.shape)

圖像資料: (1783, 96, 96) 
關鍵點資料: (1783, 1)


In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


from keras import backend as K
if K.image_dim_ordering() == 'th':
    X_train = imgs_train.reshape(imgs_train.shape[0], 1, img_rows, img_cols)
    X_test = imgs_test.reshape(imgs_test.shape[0], 1, img_rows, img_cols)
    input_shape = (points_train.shape[-1], img_rows, img_cols)
else:
    X_train = imgs_train.reshape(imgs_train.shape[0], img_rows, img_cols, 1)
    X_test = imgs_test.reshape(imgs_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, points_test.shape[-1])

In [9]:
# 定義人臉關鍵點檢測網路
model = Sequential()

# 定義神經網路的輸入, hidden layer 以及輸出
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', input_shape=(96, 96, 1)))
model.add(Activation('relu'))

model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(filters=16, kernel_size=(3,3),padding='same', activation='relu')) 
model.add(Conv2D(filters=16, kernel_size=(3,3),activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),padding='same', activation='relu')) 
model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),padding='same', activation='relu')) 
model.add(Conv2D(filters=128, kernel_size=(3,3),activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(30)) 

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 64)        640       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 96, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 94, 94, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 16)        4624      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 45, 45, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 16)        0         
__________

In [10]:
model.compile(loss='mean_squared_error', optimizer='adam')